# Deep Learning : Pytorch

- GPU 사용을 위한 CUDA 버전에 따른 설치는 여기서 확인: https://pytorch.org/get-started/previous-versions/

## 1. Pytroch Autograd

In [ ]:
!pip install torch


In [1]:
import torch
torch.__version__

'2.0.1+cu117'

In [2]:
x = torch.randn(3)
y = x + 2

In [3]:
x

tensor([-0.4798, -0.9456,  0.3479])

In [4]:
y

tensor([1.5202, 1.0544, 2.3479])

In [5]:
z = y * y * 3
z

tensor([ 6.9331,  3.3353, 16.5379])

In [6]:
z.mean()

tensor(8.9354)

In [9]:
z.mean().item()

8.935440063476562

In [10]:
# requires_grad를 지정
a = torch.rand(2, 2, requires_grad=True)
a

tensor([[0.6245, 0.5441],
        [0.9295, 0.7642]], requires_grad=True)

In [11]:
a.requires_grad

True

In [12]:
a.detach()

tensor([[0.6245, 0.5441],
        [0.9295, 0.7642]])

In [17]:
# torch.no_grad() 
with torch.no_grad():
    print(x**2)

tensor([0.2302, 0.8942, 0.1210])


In [18]:
weight = torch.ones(4, requires_grad=True)

for epoch in range(5):
    model_out = (weight*3).sum()
    model_out.backward()
    
    print(weight.grad)
    
    with torch.no_grad():
        weight-= 0.1 * weight.grad
        
    weight.grad.zero_()
    
print(weight)
print(model_out)

tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([-0.5000, -0.5000, -0.5000, -0.5000], requires_grad=True)
tensor(-2.4000, grad_fn=<SumBackward0>)


## 2. Backpropagation

In [27]:
x = torch.tensor(1.0)
y = torch.tensor(2.0)

In [28]:
w = torch.tensor(1.0, requires_grad=True)

In [29]:
w

tensor(1., requires_grad=True)

In [30]:
y_predict = w * x
loss = (y - y_predict)**2
print(loss)

tensor(1., grad_fn=<PowBackward0>)


In [31]:
loss.backward()
w.grad

tensor(-2.)

In [33]:
with torch.no_grad():
    w -= 0.01 * w.grad
w.grad.zero_()

tensor(0.)

## 3. Gradient Descent

### 3.1 Manual Build

In [56]:
import numpy as np

X = np.array([1,2,3,4], dtype=np.float32)
Y = np.array([2,4,6,8], dtype=np.float32)

w = 0.0

In [57]:
def forward(x):
    return w*x

In [58]:
def loss(y, y_pred):
    return ((y_pred - y)**2).mean()

In [59]:
def gradient(x, y, y_pred):
    return np.dot(2*x, y_pred-y).mean()
print(forward(5))

0.0


In [60]:
# training
learning_rate = 0.01
n_iters = 20

for epoch in range(n_iters):
    # foward pass
    y_pred = forward(X)
    
    # loss
    l = loss(Y, y_pred)
    
    # gradients 계산
    dw = gradient(X, Y, y_pred)
    
    # update weight
    w -= learning_rate * dw
    
    if epoch % 2 == 0:
        print(f'epoch {epoch+1}: w = {w:.3f}, loss = {l:.4f}')
print(f'predict after training: f(5) = {forward(5):.3f}')

epoch 1: w = 1.200, loss = 30.0000
epoch 3: w = 1.872, loss = 0.7680
epoch 5: w = 1.980, loss = 0.0197
epoch 7: w = 1.997, loss = 0.0005
epoch 9: w = 1.999, loss = 0.0000
epoch 11: w = 2.000, loss = 0.0000
epoch 13: w = 2.000, loss = 0.0000
epoch 15: w = 2.000, loss = 0.0000
epoch 17: w = 2.000, loss = 0.0000
epoch 19: w = 2.000, loss = 0.0000
predict after training: f(5) = 10.000


### 3.2. Autograd

In [65]:
import torch

X = torch.tensor([1,2,3,4], dtype=torch.float32)
Y = torch.tensor([2,4,6,8], dtype=torch.float32)

w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)

In [66]:
def forward(x):
    return w*x

def loss(y, y_pred):
    return ((y_pred-y)**2).mean()

In [67]:
# torch training
learning_rate = 0.01
n_iters = 100

for epoch in range(n_iters):
    # foward predict
    y_pred = forward(X)
    
    # loss
    l = loss(Y, y_pred)
    
    # backward pass
    l.backward()
    
    # update weights
    # w.data = w.data - learning_rate*w.grad
    with torch.no_grad():
        w -= learning_rate * w.grad
        
    # update 후에 w를 zero화 시킨다
    w.grad.zero_()
    
    if epoch % 10 == 0:
        print(f'epoch {epoch+1}: w={w.item():.3f}, loss={l.item():4f}')
        
print(f'Prediction after trining: f(5)={forward(5).item():.3f}')

epoch 1: w=0.300, loss=30.000000
epoch 11: w=1.665, loss=1.162786
epoch 21: w=1.934, loss=0.045069
epoch 31: w=1.987, loss=0.001747
epoch 41: w=1.997, loss=0.000068
epoch 51: w=1.999, loss=0.000003
epoch 61: w=2.000, loss=0.000000
epoch 71: w=2.000, loss=0.000000
epoch 81: w=2.000, loss=0.000000
epoch 91: w=2.000, loss=0.000000
Prediction after trining: f(5)=10.000


## 4. Loss Function & Optimizer

In [71]:
import torch
import torch.nn as nn

X = torch.tensor([1,2,3,4], dtype=torch.float32)
Y = torch.tensor([2,4,6,8], dtype=torch.float32)

w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)

In [72]:
def forward(x):
    return w*x

In [73]:
learning_rate = 0.01
n_iters = 100

In [74]:
# loss function and optimizer
loss = nn.MSELoss()
optimizer = torch.optim.SGD([w], lr=learning_rate)

In [75]:
for epoch in range(n_iters):
    # foward predict
    y_pred = forward(X)
    
    # loss
    l = loss(Y, y_pred)
    
    # backward pass
    l.backward()
    
    # update weights
    optimizer.step()
        
    # update 후에 w를 zero화 시킨다
    optimizer.zero_grad()
    
    if epoch % 10 == 0:
        print('epoch', epoch+1, ':w=', w, 'loss=', l)
        
print(f'Prediction after trining: f(5)={forward(5).item():.3f}')

epoch 1 :w= tensor(0.3000, requires_grad=True) loss= tensor(30., grad_fn=<MseLossBackward0>)
epoch 11 :w= tensor(1.6653, requires_grad=True) loss= tensor(1.1628, grad_fn=<MseLossBackward0>)
epoch 21 :w= tensor(1.9341, requires_grad=True) loss= tensor(0.0451, grad_fn=<MseLossBackward0>)
epoch 31 :w= tensor(1.9870, requires_grad=True) loss= tensor(0.0017, grad_fn=<MseLossBackward0>)
epoch 41 :w= tensor(1.9974, requires_grad=True) loss= tensor(6.7705e-05, grad_fn=<MseLossBackward0>)
epoch 51 :w= tensor(1.9995, requires_grad=True) loss= tensor(2.6244e-06, grad_fn=<MseLossBackward0>)
epoch 61 :w= tensor(1.9999, requires_grad=True) loss= tensor(1.0176e-07, grad_fn=<MseLossBackward0>)
epoch 71 :w= tensor(2.0000, requires_grad=True) loss= tensor(3.9742e-09, grad_fn=<MseLossBackward0>)
epoch 81 :w= tensor(2.0000, requires_grad=True) loss= tensor(1.4670e-10, grad_fn=<MseLossBackward0>)
epoch 91 :w= tensor(2.0000, requires_grad=True) loss= tensor(5.0768e-12, grad_fn=<MseLossBackward0>)
Prediction

## 5. Advanced Training

In [104]:
import torch
import torch.nn as nn

X = torch.tensor([[1],[2],[3],[4]], dtype=torch.float32)
Y = torch.tensor([[2],[4],[6],[8]], dtype=torch.float32)

n_samples, n_features = X.shape

In [105]:
X_test = torch.tensor([5], dtype=torch.float32)
X_test

tensor([5.])

In [106]:
input_size = X.shape[1]
output_size = Y.shape[1]

In [107]:
# model build
model = nn.Linear(input_size, output_size)

In [108]:
class LinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        # define layers
        self.lin = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        return self.lin(x)

In [109]:
model = LinearRegression(input_size, output_size)

In [110]:
learning_rate=0.01
n_iters=100

# loss function and optimizer
loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [111]:
for epoch in range(n_iters):
    # foward predict
    y_pred = model(X)
    
    # loss
    l = loss(Y, y_pred)
    
    # backward pass
    l.backward()
    
    # update weights
    optimizer.step()
        
    # update 후에 w를 zero화 시킨다
    optimizer.zero_grad()
    
    if epoch % 10 == 0:
        w, b = model.parameters()
        print('epoch', epoch+1, ':w=', w[0].item(), 'loss=', l)
        
print(f'Prediction after trining: f(X_test)={model(X_test).item():.3f}')

epoch 1 :w= -0.05885088071227074 loss= tensor(35.3990, grad_fn=<MseLossBackward0>)
epoch 11 :w= 1.3112915754318237 loss= tensor(1.1753, grad_fn=<MseLossBackward0>)
epoch 21 :w= 1.5422389507293701 loss= tensor(0.2748, grad_fn=<MseLossBackward0>)
epoch 31 :w= 1.5896360874176025 loss= tensor(0.2372, grad_fn=<MseLossBackward0>)
epoch 41 :w= 1.6072065830230713 loss= tensor(0.2229, grad_fn=<MseLossBackward0>)
epoch 51 :w= 1.6196857690811157 loss= tensor(0.2099, grad_fn=<MseLossBackward0>)
epoch 61 :w= 1.6310608386993408 loss= tensor(0.1977, grad_fn=<MseLossBackward0>)
epoch 71 :w= 1.641981601715088 loss= tensor(0.1862, grad_fn=<MseLossBackward0>)
epoch 81 :w= 1.6525605916976929 loss= tensor(0.1753, grad_fn=<MseLossBackward0>)
epoch 91 :w= 1.662824273109436 loss= tensor(0.1651, grad_fn=<MseLossBackward0>)
Prediction after trining: f(X_test)=9.324
